# Redução e Transformação de DDs

O objetivo principal desta simulação no google Colab é apresentar e discutir estratégias para redução de dados com o objetivo de reduzir a dimensionalidade de conjunto de dados.

A redução é importante qdo se deseja diminuir:


*   qt// de objetos (instâncias na base de DDs)
*   qt// de atributos

Por que realizar este procedimento, etapa?
* p/ diminuir o impacto nos resultados dos algoritmos;
* porque esse número elevado de instancias e atributos tem um custo computacional (aumenta a complexidade de processa/o);
* são gerados modelos complexos

A redução pode acontecer no nível de objetos (instâncias ou atributos).

### Seleção de atributos

Seria a remoção de atributos *irrelevantes* ou *redundantes*

Exemplos:
* Identificador de registro "id";
* Nome x usuário;
* Data de nascimento x idade.


## Compressão de Atributos

* Compactação dos DDs (ex.: diminuição da precisão de coordenadas GPS)
* Codificação/Transformação de atributos
    Análise dos Componentes Principais "PCA" - Principal Componente Analysis

PCA: Verifica correlação entre atributos
ex.: renda x valor de imóvel => componente

* Número de componentes acaba sendo < n.° atributos
* Ordenados por variância: preservar as características dos DDs que contribuem para a maior variância, ou seja, apresentam o maior peso na variação de um atributo alvo.







Neste exercício, será feito o pré-processamento de uma base de dados, aplicando alguns dos conceitos vistos na aula da semana 2.

A base de dados de exemplo simula registros de casos de pessoas que foram infectadas com a COVID-19 no Brasil.

O dicionário de dados da base é o seguinte:

• id: campo identificador do registro;

• idade: idade da pessoa;

• uf: estado onde a pessoa foi diagnosticada;

• renda: classe social a qual a pessoa pertence, variando entre A e E;

• vacina: indica se a pessoa foi vacinada (1) ou não (0).

In [1]:
# importando as bibliotecas necessárias para análises

import pandas as pd
import numpy as np

In [2]:
# importando a base de dados via "url"

url = "https://raw.githubusercontent.com/higoramario/univesp-com360-mineracao-dados/main/dados-covid-limpeza.csv"

# fazendo a leitura com o pandas

casos_covid = pd.read_csv(url)

# lendo as primeiras 10 linhas
casos_covid.head(10)

,id,idade,uf,renda,vacina
0,1,15.0,SP,C,1
1,2,38.0,MG,NaN,0
2,3,49.0,RJ,C,0
3,4,71.0,BA,NaN,0
4,5,58.0,MG,D,1
5,6,26.0,SP,B,0
6,7,87.0,BA,NaN,1
7,8,56.0,MG,A,1
8,9,74.0,BA,NaN,0
9,10,54.0,SP,NaN,0


In [4]:
# verificar quais são os tipos de dados de cada coluna (campo textual = 'object')
casos_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      50 non-null     int64  
 1   idade   30 non-null     float64
 2   uf      29 non-null     object 
 3   renda   31 non-null     object 
 4   vacina  50 non-null     int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 2.1+ KB


In [5]:
# A análise descritiva (função describe()) desses dados olhando a distribuição dos dados.

# Por exemplo, a média de idade é 48 anos, com
# desvio-padrão de 22 anos. Já para os dados nominais (uf) e ordinais (renda), não
# temos esses valores. Como os dados do atributo vacina são binários, a média
# também não faz sentido

casos_covid.describe()

,id,idade,vacina
count,50.00000,30.000000,50.00000
mean,25.50000,48.666667,0.30000
std,14.57738,22.197131,0.46291
min,1.00000,14.000000,0.00000
25%,13.25000,33.500000,0.00000
50%,25.50000,48.500000,0.00000
75%,37.75000,61.000000,1.00000
max,50.00000,87.000000,1.00000


In [6]:
# quantidade de valores distintos desses atributos com a função value_counts().

casos_covid['uf'].value_counts()

SP    9
MG    5
BA    3
RS    3
RJ    2
PE    2
sp    1
ce    1
rj    1
CE    1
AM    1
Name: uf, dtype: int64

In [8]:
'''No atributo uf, há mais registros de São Paulo. Além disso, há registros de São
Paulo e do Ceará em maiúsculo e minúsculo. Vamos fazer a padronização dos
dados da coluna uf usando o método upper(). Isso vai mudar a contagem de
registros por estado.'''

casos_covid['uf'] = casos_covid['uf'].str.upper()
casos_covid['uf'].value_counts()


SP    10
MG     5
RJ     3
BA     3
RS     3
CE     2
PE     2
AM     1
Name: uf, dtype: int64

In [10]:
# valores de renda e vacina.

casos_covid['renda'].value_counts()



C    14
D     7
B     4
A     3
E     3
Name: renda, dtype: int64

In [11]:
# vacina
casos_covid['vacina'].value_counts()

0    35
1    15
Name: vacina, dtype: int64

In [ ]:
# Podemos observar algumas características desses dados:
# • 35 das 50 pessoas diagnosticadas com COVID-19 não tomaram vacina.
# • Há mais pessoas da classe C entre os infectados

In [12]:
"""
Pré-processamentos para imputar os valores ausentes na base de dados usando as
características dos dados. Na coluna idade vamos usar a média para imputar
os valores ausentes.

Usar a função fillna(), que preenche os valores nulos a partir de
algum método, nesse caso, arredondando a média de idade, já que é um valor
inteiro.

Olhando a quantidade de registros por valores distintos, vemos o resultado
da modificação.

"""
casos_covid['idade'].fillna(round(casos_covid['idade'].mean()), inplace=True)
casos_covid['idade'].value_counts()

49.0    21
84.0     2
26.0     2
87.0     2
54.0     2
38.0     2
61.0     2
35.0     1
33.0     1
45.0     1
57.0     1
41.0     1
20.0     1
48.0     1
37.0     1
15.0     1
68.0     1
14.0     1
22.0     1
74.0     1
56.0     1
58.0     1
71.0     1
17.0     1
Name: idade, dtype: int64

In [13]:
"""
Para preencher o campo uf, vamos usar o método de imputar o valor de acordo
com a última observação. Para isso, vamos escolher a opção ffill na função fillna.
Olhando a contagem de registros por estado, podemos ver que a proporção foi
minimamente mantida.
"""

casos_covid['uf'].fillna(method="ffill",inplace=True)
casos_covid['uf'].value_counts()

SP    20
RJ     6
RS     6
MG     5
PE     5
CE     4
BA     3
AM     1
Name: uf, dtype: int64

In [14]:
"""fazer a imputação de renda usando o método de imputar o valor de
acordo com a observação seguinte, que preenche os valores ausentes a partir da
observação seguinte válida (ou para trás). Para isso, vamos escolher a opção bfill
na função fillna.
"""

casos_covid['renda'].fillna(method="bfill",inplace=True)
casos_covid['renda'].value_counts()

C    22
D    12
B     6
A     5
E     5
Name: renda, dtype: int64

In [15]:
""" Feito um processo de limpeza de dados, que pode ser verificado
usando o método info() ou olhando o dataset inteiro.
"""

casos_covid.info()
casos_covid.head(len(casos_covid))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      50 non-null     int64  
 1   idade   50 non-null     float64
 2   uf      50 non-null     object 
 3   renda   50 non-null     object 
 4   vacina  50 non-null     int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 2.1+ KB


,id,idade,uf,renda,vacina
0,1,15.0,SP,C,1
1,2,38.0,MG,C,0
2,3,49.0,RJ,C,0
3,4,71.0,BA,D,0
4,5,58.0,MG,D,1
5,6,26.0,SP,B,0
6,7,87.0,BA,A,1
7,8,56.0,MG,A,1
8,9,74.0,BA,C,0
9,10,54.0,SP,C,0
